

### Gridap day at "Groupe Calcul"

# Exercise 0


The following parallel code computes the Poisson equation with Dirichlet boundary conditions on the unit cube. It measures the time of the main phases of the algorithm and saves the results to a file. `n` is the number of cells in each direction. `np` is the number of parts in each direction. `nruns` is the number of times the algorithm is run. We need to run several times to obtain reliable time measures.

The exercise consists in running this example in parallel using MPI on a system of your choice. In a laptop you should be able to run for a problem of size of `n=(50,50,50)` for `np=(1,1,1)`, `np=(2,1,1)` and `np=(2,2,1)` and see some speedup as the number of MPI rank increases. In a cluster, you should be able tu run for much larger problem sizes and more MPI ranks.

Main steps.

1. System Image. Generate a package e.g. with `pkg> generate MyPackage` and copy the main function below into the package. Generate a system image of this package using PackageCompiler.jl. The cell starting with `# compile.jl` contains the code needed to generate the system image. Copy the contents of the cell starting with `# warmup.jl` into a file called `warmup.jl` and use it as a warm-up script. This will take several minutes to finish.

3. Run the code in parallel with a command like `mpiexec -np 4 julia --project=. -J MySysImg.so driver.jl`. Don't forget to use the system image. The version of `mpiexec` needs to match with the installation being used by MPI.jl. See MPI.jl (v0.19) for details. The contents of file `driver.jl` are given in a cell below.



In [ ]:
using Gridap
using GridapDistributed
using GridapPETSc
using PartitionedArrays
using FileIO

In [ ]:
function main(parts,cells,i,options)
    timer = PTimer(parts)
    u(x) = sum(x)
    f(x) = -Δ(u,x)
    g = u
    k = 1
    domain = (0,1,0,1,0,1)
    tic!(timer,barrier=true)
    model = CartesianDiscreteModel(parts,domain,cells)
    toc!(timer,"mesh")
    tic!(timer,barrier=true)
    reffe = ReferenceFE(lagrangian,Float64,k)
    Vh = TestFESpace(model,reffe,dirichlet_tags="boundary")
    Uh = TrialFESpace(Vh,g)
    toc!(timer,"space")
    tic!(timer,barrier=true)
    Ω = Interior(model)
    dΩ = Measure(Ω,2*k)
    a(u,v) = ∫(∇(u)⋅∇(v))dΩ
    l(v) = ∫(v*f)dΩ
    op = AffineFEOperator(a,l,Uh,Vh)
    toc!(timer,"assembly")
    uh = GridapPETSc.with(args=split(options)) do
        solver = PETScLinearSolver()
        tic!(timer,barrier=true)
        uh = solve(solver,op)
        toc!(timer,"solver")
        uh
    end
    tic!(timer,barrier=true)
    e = u - uh
    contrib_h1 = ∫(∇(e)⋅∇(e) + e*e )dΩ
    h1 = sqrt(sum(contrib_h1))
    toc!(timer,"error")
    display(timer)
    map_main(timer.data) do data
        ns = join(map(c->"_"*string(c),cells))
        nps = join(map(c->"_"*string(c),size(parts)))
        name = "results_run$(i)$(ns)$(nps).jld2"
        output = Dict{String,Any}(data)
        output["h1"] = h1
        save(name,output)
    end
    nothing
end

In [ ]:
# compile.jl
using PackageCompiler
create_sysimage([:Example1],
  sysimage_path=joinpath(@__DIR__,"MyPackage.so"),
  precompile_execution_file=joinpath(@__DIR__,"warmup.jl"))

In [ ]:
# warmup.jl
using PartitionedArrays
using MyPackage
backend = MPIBackend()
n = (10,10,10)
np = (1,1,1)
options = "-pc_type gamg -ksp_type cg -ksp_converged_reason -ksp_rtol 1.0e-12"
MyPackage.main(parts,n,1,options)

In [ ]:
# driver.jl
using PartitionedArrays
using MyPackage
backend = MPIBackend()
n = (50,50,50)
np = (2,2,1)
nruns = 3
options = "-pc_type gamg -ksp_type cg -ksp_converged_reason -ksp_rtol 1.0e-12"
prun(backend,np) do parts
    for i in 1:nruns
        MyPackage.main(parts,n,i,options)
    end
end